In [1]:
import numpy as np
import os
from PIL import Image, ImageOps
import numbers
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dataset & Data Loader

In [2]:
class ResizeImage():
    def __init__(self, size):
        if isinstance(size, int):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, img):
        th, tw = self.size
        return img.resize((th, tw))

In [3]:
class PlaceCrop(object):
    """Crops the given PIL.Image at the particular index.
    Args:
        size (sequence or int): Desired output size of the crop. If size is an
            int instead of sequence like (w, h), a square crop (size, size) is
            made.
    """

    def __init__(self, size, start_x, start_y):
        if isinstance(size, int):
            self.size = (int(size), int(size))
        else:
            self.size = size
        self.start_x = start_x
        self.start_y = start_y

    def __call__(self, img):
        """
        Args:
            img (PIL.Image): Image to be cropped.
        Returns:
            PIL.Image: Cropped image.
        """
        th, tw = self.size
        return img.crop((self.start_x, self.start_y, self.start_x + tw, self.start_y + th))

In [5]:
batch_size = 64
crop_size = 224

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(crop_size, scale=(0.7, 1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_set = datasets.ImageFolder("data/ant-1.6-preprop/", transform = train_transform)
trainloader = DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers=4)

test_set = datasets.ImageFolder("data/poi-3.0-preprop/", transform = test_transform)
testloader = DataLoader(test_set, batch_size = batch_size, shuffle = True)

In [6]:
label_data = train_set.classes
label_data

['buggy', 'clean']

# Arsitektur & Config

In [7]:
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn, optim

In [14]:
class ModelResNet(nn.Module):
    def __init__(self, output_size):
        super().__init__()
        self.model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.freeze()
        self.model.fc = nn.Sequential(
            nn.Linear(2048, 2),
            nn.LogSoftmax(1)
        )
        
    def forward(self, x):
        return self.model(x)
    
    def freeze(self):
        for param in self.model.parameters():
            param.requires_grad = False
            
    def unfreeze(self):
        for param in self.model.parameters():
            param.requires_grad = True

In [15]:
config = set_config({
    "output_size": len(label_data),
    "batch_size": batch_size,
    "crop_size": crop_size
})

# Phase 1 : Adaptation (Ir standard + patience kecil)

In [16]:
model = ModelResNet(config.output_size).to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
callback = Callback(model, config, early_stop_patience=2, outdir="model")

In [17]:
from tqdm.auto import tqdm

def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    elif mode == "test":
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model (feature) 
        loss = criterion (output, target)
        if mode == "train":
            loss.backward() 
            optimizer.step() 
            optimizer.zero_grad()
        cost + loss.item() * feature.shape [0]
        correct += (output.argmax(1) == target).sum().item()
    cost = cost/len(dataset) 
    acc = correct/len(dataset) 
    return cost, acc

In [ ]:
while True:
    train_cost, train_score = loop_fn("train", train_set, trainloader, model, criterion, optimizer, device) 
    with torch.no_grad():
        test_cost, test_score = loop_fn("test", test_set, testloader, model, criterion, optimizer, device)
        
    # Logging 
    callback.log(train_cost, test_cost, train_score, test_score)
    
    # Checkpoint 
    callback.save_checkpoint()
    
    # Runtime Plotting 
    callback.cost_runtime_plotting() 
    callback.score_runtime_plotting()
    
    # Early Stopping 
    if callback.early_stopping (model, monitor="test_score"):
        callback.plot_cost() 
        callback.plot_score() 
        break

Train:   0%|          | 0/6 [00:00<?, ?it/s]

Test:   0%|          | 0/7 [00:00<?, ?it/s]


Epoch     1
Train_cost  = 0.0000 | Test_cost  = 0.0000 | Train_score = 0.7249 | Test_score = 0.3626 |


Train:   0%|          | 0/6 [00:01<?, ?it/s]

Test:   0%|          | 0/7 [00:00<?, ?it/s]

...

Train:   0%|          | 0/6 [00:00<?, ?it/s]

Test:   0%|          | 0/7 [00:00<?, ?it/s]


Epoch     4
Train_cost  = 0.0000 | Test_cost  = 0.0000 | Train_score = 0.8195 | Test_score = 0.4665 |


Train:   0%|          | 0/6 [00:01<?, ?it/s]

Test:   0%|          | 0/7 [00:00<?, ?it/s]